<a href="https://colab.research.google.com/github/KIQVinicius/Data-Science/blob/master/AnomaliasJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
from datetime import timedelta

In [73]:
import numpy as np
from datetime import date

In [91]:
dataset = pd.read_csv("MIPS1.csv")
dataset = dataset.head(50000)

In [92]:
porcentagem = float(dataset['Porcentagem'][1])/100 + 1
dataset.columns = ['Index', 'JOBNAME', 'DATE', 'Valor_MIPS', 'Porcentagem', 'Chave']

In [93]:
dataset = dataset.drop(columns=["Porcentagem", "Chave"], axis=1)

In [94]:
dataset['DATE'] = pd.to_datetime(dataset['DATE'])

In [95]:
first_day = dataset['DATE'].min()
last_day = dataset['DATE'].max()

In [96]:
first_day

Timestamp('2020-01-04 00:00:00')

In [97]:
last_day

Timestamp('2020-05-04 00:00:00')

In [98]:
dt = last_day.date() - first_day.date()
dt.days

121

In [99]:
weeks = int(dt.days/7)
weeks

17

In [100]:
months = int(dt.days/30)
months

4

In [101]:
data = first_day

In [102]:
frequencia = pd.DataFrame(dataset['JOBNAME'].value_counts())
frequencia.reset_index(inplace=True)
frequencia.columns = ['JOBNAME', 'Frequencia']

In [103]:
frequencia = frequencia.query("Frequencia > 1")

In [104]:
dataset = dataset[dataset['JOBNAME'].isin(frequencia['JOBNAME'])]
dataset.head()

,Index,JOBNAME,DATE,Valor_MIPS
0,0,FK20OK,2020-03-25,0.03
1,1,FK20PA1,2020-03-25,0.06
2,2,FK20PA2,2020-03-25,0.26
3,3,FK20PA3,2020-03-25,0.15
4,4,FK20PA4,2020-03-25,0.04


In [105]:
pd.DataFrame(dataset['JOBNAME'].value_counts())
dataset = dataset.drop(columns=["Index"], axis=1)

# Anomalia


In [106]:
def DetectarAnomalia(dataset, intervalo):
  if intervalo == "Mensal":
    periodo = months
    intervalo_de_dias = 30
  else:
    periodo = weeks
    intervalo_de_dias = 7
  data = first_day
  tabela_anomalia = dataset.iloc[0:0]
  dataset_semana_anterior = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = intervalo_de_dias)) + "' ")
  data = data+ timedelta(days = intervalo_de_dias)
  dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['Valor_MIPS']>0].groupby('JOBNAME')
  dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
  dataset_semana_anterior.reset_index(inplace =True)
  data = data+ timedelta(days = intervalo_de_dias)
  for i in range(0, periodo, 1):
    dataset_semana = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = intervalo_de_dias)) + "' ")
    #dataset = dataset.query("DATE >= " + " '"+ str(data+ timedelta(days = 30))+ "' ")
    print("Anomalias de " + str(data)[0:10] + " a "+ str(data + timedelta(days = intervalo_de_dias) )[0:10]+": "  + str(tabela_anomalia.shape[0]))
    print("----------------------")
    #dataset_semana = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
    #dataset_semana = pd.DataFrame(dataset_semana['Valor_MIPS'].mean())
    for index, row in dataset_semana_anterior.iterrows():
      tabela_anomalia = tabela_anomalia.append(dataset_semana.query("JOBNAME == "+"'"+row['JOBNAME']+"'" + " and Valor_MIPS > "+ str(float(row['Valor_MIPS']) * porcentagem)))
    dataset_semana_anterior = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
    dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
    dataset_semana_anterior.reset_index(inplace=True)
    data = data+ timedelta(days = intervalo_de_dias)
  tabela_anomalia['Anomalia'] = intervalo
  return tabela_anomalia


In [109]:
tabela_mensal = DetectarAnomalia(dataset, "Mensal")

Anomalias de 2020-03-04 a 2020-04-03: 0
----------------------
Anomalias de 2020-04-03 a 2020-05-03: 7768
----------------------
Anomalias de 2020-05-03 a 2020-06-02: 9192
----------------------
Anomalias de 2020-06-02 a 2020-07-02: 9631
----------------------


In [110]:
tabela_semanal = DetectarAnomalia(dataset, "Semanal")

Anomalias de 2020-01-18 a 2020-01-25: 0
----------------------
Anomalias de 2020-01-25 a 2020-02-01: 0
----------------------
Anomalias de 2020-02-01 a 2020-02-08: 0
----------------------
Anomalias de 2020-02-08 a 2020-02-15: 0
----------------------
Anomalias de 2020-02-15 a 2020-02-22: 0
----------------------
Anomalias de 2020-02-22 a 2020-02-29: 0
----------------------
Anomalias de 2020-02-29 a 2020-03-07: 0
----------------------
Anomalias de 2020-03-07 a 2020-03-14: 0
----------------------
Anomalias de 2020-03-14 a 2020-03-21: 0
----------------------
Anomalias de 2020-03-21 a 2020-03-28: 0
----------------------
Anomalias de 2020-03-28 a 2020-04-04: 0
----------------------
Anomalias de 2020-04-04 a 2020-04-11: 3744
----------------------
Anomalias de 2020-04-11 a 2020-04-18: 5136
----------------------
Anomalias de 2020-04-18 a 2020-04-25: 5136
----------------------
Anomalias de 2020-04-25 a 2020-05-02: 5136
----------------------
Anomalias de 2020-05-02 a 2020-05-09: 5136


In [123]:
tabela_final = tabela_semanal.iloc[0:0]
tabela_final = tabela_final.append(tabela_mensal)
print(tabela_final.shape)
tabela_final = tabela_final.append(tabela_semanal)
print(tabela_final.shape)

(9631, 4)
(14767, 4)


In [124]:
tabela_final

,JOBNAME,DATE,Valor_MIPS,Anomalia
23468,ANTMAIN,2020-03-31,0.14,Mensal
2449,AOPD,2020-03-26,1.12,Mensal
7777,AOPD,2020-03-27,1.24,Mensal
2450,APPC,2020-03-26,0.96,Mensal
2451,ARCABKUP,2020-03-26,57.77,Mensal
...,...,...,...,...
47274,TWSRPTJB,2020-04-04,0.02,Semanal
47276,VLF,2020-04-04,0.98,Semanal
47278,VTFM,2020-04-04,1198.82,Semanal
47285,WEKAMSTR,2020-04-04,2.06,Semanal
